In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%matplotlib inline 

In [19]:
from yaml import load as yaml_load
import os
import sys
import json
sys.path.append('../')
import findspark
findspark.init()
import pyspark
from pyspark.sql.types import LongType
from pyspark.sql.functions import isnan, when, count, col, isnull
from pyspark.sql import SparkSession
import pyspark.sql.functions as func
from pyspark.ml import Pipeline
from pyspark.ml.regression import LinearRegression, GBTRegressor
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.feature import StandardScaler, VectorAssembler, VectorIndexer, StandardScaler
from pyspark.sql.functions import *
from pyspark.sql.functions import pandas_udf, PandasUDFType
import pandas as pd
from pyspark.sql.functions import col, pandas_udf, udf
from pyspark.sql.types import LongType, StringType, FloatType, IntegerType, DoubleType
from pyspark.ml.feature import OneHotEncoderEstimator, StringIndexer, VectorAssembler
from pyspark.ml.evaluation import BinaryClassificationEvaluator, MulticlassClassificationEvaluator
import re
from pyspark.ml.classification import RandomForestClassifier, RandomForestClassificationModel, GBTClassifier,OneVsRest, OneVsRestModel
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer, IndexToString, RFormula
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
spark = SparkSession.builder.master("local[*]").appName("Data cleaning").getOrCreate()

In [4]:
from src.data.make_dataset import LoadDataframe
from src.features.build_features import extract_features_classification
from src.models.train_model import model_classification

In [5]:
def _load_config_file(config_file):
    """
    Load configuration file
    :param config_file: is the configuration file
    :return: configuration
    :rtype: dict
    """
    with open(config_file) as yml_config:
        return yaml_load(yml_config)

def _build_configuration(config_file):
    """
    Build the operation configuration dict
    :param config_file: is the path to the yaml config_file
    :type: string
    :return: config: global configuration
    :rtype dict
    """
    # yaml config
    config = _load_config_file(config_file)
    return config
def visualisation_prediction(y_test, y_pred):
    import matplotlib
    import matplotlib.pyplot as plt
    matplotlib.rc('xtick', labelsize=30) 
    matplotlib.rc('ytick', labelsize=30) 
    fig, ax = plt.subplots(figsize=(50, 40))
    plt.style.use('ggplot')
    plt.plot(y_pred, y_test, 'ro')
    plt.xlabel('Predicted Crime', fontsize = 30)
    plt.ylabel('Actual Crime', fontsize = 30)
    plt.title('Predicted Y (Crimes) to the Actual Y (Crimes)', fontsize = 30)
    ax.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'k--', lw=4)
    

config_file = "/home/ml/Documents/Crime_Chigaco_Spark/config/config.yml"
config = _build_configuration(config_file)

In [6]:
%%time
obj_df_loaded = LoadDataframe(config, '2013', '2014')


CPU times: user 349 µs, sys: 45 µs, total: 394 µs
Wall time: 400 µs


In [7]:
df_crimes_socio = obj_df_loaded.df_crime_socio()
df_temp = obj_df_loaded.df_temperature()
df_sky  = obj_df_loaded.df_sky()

/usr/lib/spark/spark-2.4.0-bin-hadoop2.7/python/pyspark/sql/dataframe.py:1793: UserWarning: to_replace is a dict and value is not None. value will be ignored.
  warnings.warn("to_replace is a dict and value is not None. value will be ignored.")


In [8]:
obj_extract_features_classification = extract_features_classification(config, df_crimes_socio, df_temp, df_sky)

In [9]:
df_ml = obj_extract_features_classification.extract_feature()

In [10]:
obj_model_classification = model_classification(config, df_ml)

In [12]:
%%time
rf_model = obj_model_classification.train_RF()

CPU times: user 4.86 s, sys: 1.24 s, total: 6.1 s
Wall time: 54min 23s


In [13]:
df_test = obj_model_classification.df_test()

In [15]:
df_prediction = rf_model.transform(df_test)

In [18]:
df_prediction.select('primary_type','label', 'prediction','predictedLabel').limit(10).toPandas()

,primary_type,label,prediction,predictedLabel
0,ASSAULT_BATTERY,1.0,1.0,ASSAULT_BATTERY
1,NARCOTICS,2.0,1.0,ASSAULT_BATTERY
2,THEFT_ROBBERY_BURGLARY,0.0,0.0,THEFT_ROBBERY_BURGLARY
3,THEFT_ROBBERY_BURGLARY,0.0,0.0,THEFT_ROBBERY_BURGLARY
4,THEFT_ROBBERY_BURGLARY,0.0,0.0,THEFT_ROBBERY_BURGLARY
5,THEFT_ROBBERY_BURGLARY,0.0,0.0,THEFT_ROBBERY_BURGLARY
6,ASSAULT_BATTERY,1.0,0.0,THEFT_ROBBERY_BURGLARY
7,THEFT_ROBBERY_BURGLARY,0.0,0.0,THEFT_ROBBERY_BURGLARY
8,NARCOTICS,2.0,0.0,THEFT_ROBBERY_BURGLARY
9,THEFT_ROBBERY_BURGLARY,0.0,0.0,THEFT_ROBBERY_BURGLARY
